In [ ]:
_base_ = [
    '../_base_/models/faster_rcnn_r50_fpn.py',
    '../_base_/datasets/voc0712.py',
    '../_base_/schedules/schedule_1x.py', '../_base_/default_runtime.py'
]
model = dict(roi_head=dict(bbox_head=dict(num_classes=20)))
# 使用VOC2007和VOC2012数据集
data = dict(
    train=dict(
        type='RepeatDataset',
        times=3,
        dataset=dict(
            type='VOCDataset',
            ann_file=[
                'data/VOCdevkit/VOC2007/ImageSets/Main/trainval.txt',
                'data/VOCdevkit/VOC2012/ImageSets/Main/trainval.txt'
            ],
            img_prefix=['data/VOCdevkit/VOC2007/', 'data/VOCdevkit/VOC2012/']
        )
    ),
    val=dict(
        type='VOCDataset',
        ann_file='data/VOCdevkit/VOC2007/ImageSets/Main/val.txt',
        img_prefix='data/VOCdevkit/VOC2007/'),
    test=dict(
        type='VOCDataset',
        ann_file='data/VOCdevkit/VOC2007/ImageSets/Main/test.txt',
        img_prefix='data/VOCdevkit/VOC2007/'))
evaluation = dict(interval=1, metric='mAP')
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_config = dict(step=[8, 16])
runner = dict(type='EpochBasedRunner', max_epochs=24)


In [ ]:
_base_ = './yolov3_d53_mstrain-608_273e.py'
# 使用VOC2007和VOC2012数据集
dataset_type = 'VOCDataset'
data_root = 'data/VOCdevkit/'

data = dict(
    train=dict(
        type='RepeatDataset',
        times=3,
        dataset=dict(
            type=dataset_type,
            ann_file=[
                data_root + 'VOC2007/ImageSets/Main/trainval.txt',
                data_root + 'VOC2012/ImageSets/Main/trainval.txt'
            ],
            img_prefix=[data_root + 'VOC2007/', data_root + 'VOC2012/']
        )
    ),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'VOC2007/ImageSets/Main/val.txt',
        img_prefix=data_root + 'VOC2007/'),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'VOC2007/ImageSets/Main/test.txt',
        img_prefix=data_root + 'VOC2007/'))
evaluation = dict(interval=1, metric='mAP')
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_config = dict(step=[8, 16])
runner = dict(type='EpochBasedRunner', max_epochs=24)


In [ ]:
import os
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector
from mmdet.models import build_detector
from mmdet.datasets import build_dataset
from mmdet.apis import train_detector, init_random_seed
from torch.utils.tensorboard import SummaryWriter
import mmcv

def main(config_file, checkpoint=None):
    cfg = Config.fromfile(config_file)
    cfg.gpu_ids = range(1)
    cfg.seed = init_random_seed(0, device='cuda')
    set_random_seed(0, deterministic=False)
    
    # Create work_dir for saving checkpoints and logs
    mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
    log_file = os.path.join(cfg.work_dir, 'train.log')
    writer = SummaryWriter(log_dir=cfg.work_dir)
    
    # Build dataset and model
    datasets = [build_dataset(cfg.data.train)]
    model = build_detector(cfg.model)
    model.CLASSES = datasets[0].CLASSES

    # Train the detector
    train_detector(model, datasets, cfg, distributed=False, validate=True, meta=dict())
    
    # Close the SummaryWriter
    writer.close()

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='Train a detector')
    parser.add_argument('config', help='train config file path')
    parser.add_argument('--checkpoint', help='checkpoint file')
    args = parser.parse_args()

    main(args.config, args.checkpoint)


In [ ]:
import os
from mmcv import Config
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

def main(config_file, checkpoint_file, img_folder, out_folder):
    cfg = Config.fromfile(config_file)
    model = init_detector(cfg, checkpoint_file, device='cuda:0')

    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

    for img_file in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_file)
        result = inference_detector(model, img_path)
        out_file = os.path.join(out_folder, img_file)
        show_result_pyplot(model, img_path, result, out_file=out_file)

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='Test a detector')
    parser.add_argument('config', help='test config file path')
    parser.add_argument('checkpoint', help='checkpoint file')
    parser.add_argument('img_folder', help='folder of images to test')
    parser.add_argument('out_folder', help='folder to save results')
    args = parser.parse_args()

    main(args.config, args.checkpoint, args.img_folder, args.out_folder)


In [ ]:
import os
from mmcv import Config
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import cv2

def visualize_proposals(config_file, checkpoint_file, img_folder, out_folder, score_thr=0.5):
    cfg = Config.fromfile(config_file)
    model = init_detector(cfg, checkpoint_file, device='cuda:0')
    proposal_model = model.rpn_head

    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

    for img_file in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_file)
        img = cv2.imread(img_path)

        proposals = proposal_model.simple_test_rpn([img], [cfg.data.test.pipeline], rescale=False)
        proposals = proposals[0][:, :4]
        for box in proposals:
            cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

        out_file = os.path.join(out_folder, 'proposals_' + img_file)
        cv2.imwrite(out_file, img)

        result = inference_detector(model, img_path)
        show_result_pyplot(model, img_path, result, score_thr=score_thr, out_file=os.path.join(out_folder, img_file))

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='Visualize proposals and detection results')
    parser.add_argument('config', help='test config file path')
    parser.add_argument('checkpoint', help='checkpoint file')
    parser.add_argument('img_folder', help='folder of images to test')
    parser.add_argument('out_folder', help='folder to save results')
    args = parser.parse_args()

    visualize_proposals(args.config, args.checkpoint, args.img_folder, args.out_folder)


In [ ]:
import os
from mmcv import Config
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

def main(config_file, checkpoint_file, img_folder, out_folder):
    cfg = Config.fromfile(config_file)
    model = init_detector(cfg, checkpoint_file, device='cuda:0')

    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

    for img_file in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_file)
        result = inference_detector(model, img_path)
        out_file = os.path.join(out_folder, img_file)
        show_result_pyplot(model, img_path, result, out_file=out_file)

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='Test a detector on custom images')
    parser.add_argument('config', help='test config file path')
    parser.add_argument('checkpoint', help='checkpoint file')
    parser.add_argument('img_folder', help='folder of images to test')
    parser.add_argument('out_folder', help='folder to save results')
    args = parser.parse_args()

    main(args.config, args.checkpoint, args.img_folder, args.out_folder)


In [ ]:
tensorboard --logdir work_dirs